In [5]:
# Load packages
import os
import csv
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import hvplot.pandas
import requests
from config2 import geoapify_key

In [6]:
# load csv data for Housing Inventory (New Listings)
housing_inventory = Path("../Data/Housing_Inventory(New_Listings).csv")
home_inventory_df = pd.read_csv(housing_inventory)
home_inventory_df



,Date,United States,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,2016-07-01,527576.0,7756.0,1224.0,12196.0,4376.0,47100.0,13240.0,5688.0,1848.0,...,1172.0,11528.0,39148.0,5804.0,1456.0,13760.0,13136.0,1988.0,9276.0,1292.0
1,2016-08-01,470780.0,6882.0,1082.0,12218.0,3954.0,45164.0,12522.0,4924.0,1774.0,...,1100.0,11270.0,34468.0,5840.0,1372.0,12936.0,12966.0,2046.0,8210.0,1114.0
2,2016-09-01,452994.0,6674.0,898.0,12886.0,3798.0,41326.0,10848.0,5476.0,2154.0,...,1014.0,10436.0,31578.0,5036.0,1072.0,12786.0,10688.0,1734.0,7196.0,930.0
3,2016-10-01,413376.0,6096.0,788.0,13464.0,3776.0,39896.0,9768.0,4892.0,1592.0,...,860.0,9744.0,30980.0,4692.0,804.0,11100.0,9704.0,1848.0,6596.0,896.0
4,2016-11-01,376704.0,5332.0,600.0,12174.0,3464.0,34064.0,8064.0,3926.0,1378.0,...,726.0,9116.0,27726.0,4300.0,670.0,9626.0,6606.0,1556.0,6658.0,686.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2024-06-01,430076.0,7328.0,1140.0,9288.0,4356.0,32560.0,11180.0,3968.0,1304.0,...,1160.0,11792.0,42392.0,5112.0,1036.0,10944.0,11008.0,1780.0,6504.0,960.0
96,2024-07-01,405404.0,7064.0,1070.0,8602.0,4512.0,31338.0,10120.0,3576.0,1268.0,...,1100.0,10710.0,37580.0,4576.0,966.0,10482.0,10058.0,1676.0,6582.0,940.0
97,2024-08-01,383552.0,7016.0,916.0,8992.0,4356.0,30632.0,9296.0,3132.0,1300.0,...,1044.0,10696.0,35616.0,4532.0,944.0,9532.0,8768.0,1660.0,6544.0,868.0
98,2024-09-01,399750.0,6708.0,742.0,10164.0,3850.0,31988.0,9398.0,3644.0,1394.0,...,1032.0,15472.0,34858.0,4374.0,968.0,10618.0,9542.0,1680.0,6026.0,758.0


In [9]:
# Converted to a dataframe
home_inventory_df = pd.DataFrame(home_inventory_df)

# Formatted 'Date' column to datetime
home_inventory_df['Date'] = pd.to_datetime(home_inventory_df['Date'])

# Set index to 'Date'
home_inventory_df.set_index('Date', inplace=True)

# Calculated the new home listing averages for each state
st_home_inventory_df = home_inventory_df.mean()

# reset index
st_home_inventory_df = st_home_inventory_df.reset_index()

# Select columns 'State' and 'Housint Inventory Avg'
st_home_inventory_df.columns = ['State', 'Housing Inventory Avg']

# Sort values in descending order
st_home_inventory_df = st_home_inventory_df.sort_values(by='Housing Inventory Avg',ascending=False)

# Removing the United States from the dataframe
st_home_inventory_df = st_home_inventory_df.drop(0)

# states with the highest number of new home listings
st_home_inventory_df.head()

,State,Housing Inventory Avg
9,Florida,42927.84
43,Texas,36048.86
5,California,35925.14
32,New York,16700.50
10,Georgia,15975.60


In [10]:
# states with the lowest number of new home listings
st_home_inventory_df.tail()

,State,Housing Inventory Avg
41,South Dakota,981.54
2,Alaska,931.54
50,Wyoming,880.24
45,Vermont,867.96
34,North Dakota,865.34


In [11]:
# creating a list from the states
States = st_home_inventory_df["State"].tolist()


In [12]:
# created an empty list
latitudes = []
longitudes = []

# Geoapify API key
geoapify_key = geoapify_key

# Looped through each state to obtain lat and lon
for state in States:
    target_url = f"https://api.geoapify.com/v1/geocode/search?state={state}&filter=countrycode:us&format=json&apiKey={geoapify_key}"
    response = requests.get(target_url).json()
    
    try:
        # Extracted latitude and longitude
        lat = response["results"][0]["lat"]
        lon = response["results"][0]["lon"]
        latitudes.append(lat)
        longitudes.append(lon)
    except (IndexError, KeyError):
        # Handled cases where no result is found
        latitudes.append(None)
        longitudes.append(None)

# Created a DataFrame with results
geo_df = pd.DataFrame({
    "State": States,
    "Latitude": latitudes,
    "Longitude": longitudes
})

# Display results
geo_df

,State,Latitude,Longitude
0,Florida,27.756767,-81.463983
1,Texas,31.263890,-98.545612
2,California,36.701463,-118.755997
3,New York,43.156168,-75.844995
4,Georgia,32.329381,-83.113737
5,Illinois,40.079661,-89.433729
6,North Carolina,35.672964,-79.039292
7,Pennsylvania,40.969989,-77.727883
8,Ohio,40.225357,-82.688140
9,Michigan,43.621195,-84.682435


In [13]:
# Merged st_home_inventory_df with geo_df
state_visual_data = pd.merge(st_home_inventory_df,geo_df, how="outer", on="State")

# scaled down the housing inventory averages by dividing all figures by 100
# the intended purpose is so you could visual see the scale of housing inventory across the us
# without it the figures were oversized and overclustered. 
state_visual_data['Housing Inventory Avg'] = state_visual_data['Housing Inventory Avg']/100

# Displayed results
state_visual_data

,State,Housing Inventory Avg,Latitude,Longitude
0,Alabama,67.7432,33.258882,-86.829534
1,Alaska,9.3154,64.445961,-149.680909
2,Arizona,117.8540,34.395342,-111.763275
3,Arkansas,38.4172,35.204888,-92.447911
4,California,359.2514,36.701463,-118.755997
5,Colorado,98.6012,38.725178,-105.607716
6,Connecticut,44.8800,41.650020,-72.734216
7,Delaware,15.3730,38.692045,-75.401331
8,Florida,429.2784,27.756767,-81.463983
9,Georgia,159.7560,32.329381,-83.113737


In [14]:
# Configured the map plot
state_map = state_visual_data.hvplot.points(
    "Longitude",
    "Latitude",
    geo=True,
    tiles="OSM",
    frame_width=1400,
    frame_height=1155,  # Adjust frame size if needed
    size="Housing Inventory Avg",
    scale=2,
    color="State",
    hover_cols=["State", "Housing Inventory Avg", "Latitude", "Longitude"]
)

state_map.opts(
    legend_position='right',
    legend_opts={'title_text_font_size': '10pt', 'label_text_font_size': '8pt'}  # Adjust sizes
)
# Displayed map
state_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (State,Housing Inventory Avg)